# Regression with a Crab Age dataset

"The dataset for this competition (both train and test) was generated from a deep learning model trained on the Crab Age Prediction dataset."

The **aim** is to predict the **age**.

- [Kaggle challenge](https://www.kaggle.com/competitions/playground-series-s3e16/data?select=test.csv)
- [Original Dataset](https://www.kaggle.com/datasets/sidhus/crab-age-prediction)

In [ ]:
from scipy.stats import f_oneway
